In [6]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import timedelta
from datetime import datetime

def wait_for_page_load(driver, timeout=20):
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def wait_for_element(driver, clas, timeout=20):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.eventRow.flex.w-full.flex-col.text-xs'))
    )

In [2]:
Urls_ods = ['https://www.oddsportal.com/basketball/usa/nba/results/']

Nombres_ligas = ['NBA']

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(Urls_ods[0])


In [14]:
next_button = driver.find_element(By.XPATH, "//a[contains(@class, 'pagination-link') and text()='Next']")
next_button.click()

In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager

In [ ]:
def dataset_urls_ods(urls, Nombres_ligas):

    partidos = []
    ligas = []
    home_team = []
    away_team = []
    paginas2 = []
    fechas = []
    i = -1
    for url in urls:
        i += 1
        
        options = Options()
        #options.add_experimental_option("detach", True)
        options.add_argument("--disable-search-engine-choice-screen")

        #options.add_argument("--headless")
        #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

        driver.get(url)
        driver.maximize_window()

        last_height = driver.execute_script("return document.body.scrollHeigh")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-100);")
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'pagination-link') and text()='Next']"))
            )
        except Exception as e:
            prev_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[@class='pagination-link' and text()='Prev']"))
            )

        html = driver.page_source
        
        soup = BeautifulSoup(html, 'html.parser')
        
        divs = soup.find_all('div', class_='eventRow flex w-full flex-col text-xs')

        for div in divs:
            
            fecha_div = div.find('div', class_='text-black-main font-main w-full truncate text-xs font-normal leading-5')
            
            if fecha_div is not None:
                fecha_div = fecha_div.text
                fecha = fecha_div.split('-')[0].rstrip()
                if 'Today' in fecha:
                    fecha = datetime.now().strftime("%d %b %Y")
                elif 'Yesterday' in fecha:
                    fecha = (datetime.now() - timedelta(days=1)).strftime("%d %b %Y")
            divs2 = div.find_all('a', class_='next-m:flex next-m:!mt-0 ml-2 mt-2 min-h-[32px] w-full hover:cursor-pointer')

            for div2 in divs2:
                partidos.append('https://www.oddsportal.com' + div2['href'])
                equipos = div2.find_all('p', class_='participant-name truncate')
                home_team.append(equipos[0].text)
                away_team.append(equipos[1].text)
                paginas2.append(i)
                ligas.append(Nombres_ligas[i]) 
                fechas.append(fecha)
    driver.quit()
    new_data_df = pd.DataFrame()        
    new_data_df['liga'] = ligas
    new_data_df['url'] = partidos
    new_data_df['Home_team'] = home_team
    new_data_df['Away_team'] = away_team
    new_data_df['pagina'] = paginas2
    new_data_df['fecha'] = fechas

    return new_data_df

In [27]:
def scroll_to_bottom(driver, pause_time=2):
    """Desplazarse poco a poco hasta el final de la página."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Desplazarse hacia abajo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Esperar a que se cargue el contenido
        time.sleep(pause_time)
        
        # Calcular la nueva altura de la página y comparar con la última altura
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def dataset_urls_ods2(urls, Nombres_ligas):

    partidos = []
    ligas = []
    home_team = []
    away_team = []
    paginas2 = []
    fechas = []
    i = -1
    
    for url in urls:
        i += 1
        prev_button = None
        options = Options()
        #options.add_experimental_option("detach", True)
        options.add_argument("--disable-search-engine-choice-screen")

        #options.add_argument("--headless")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        #driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))

        driver.get(url)
        driver.maximize_window()

        while prev_button is None:
            

            
            scroll_to_bottom(driver,2)
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'pagination-link') and text()='Next']"))
                )
            except Exception as e:
                prev_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'pagination-link') and text()='Prev']"))
                )

            html = driver.page_source
            
            soup = BeautifulSoup(html, 'html.parser')
            
            divs = soup.find_all('div', class_='eventRow flex w-full flex-col text-xs')

            for div in divs:
                
                fecha_div = div.find('div', class_='text-black-main font-main w-full truncate text-xs font-normal leading-5')
                
                if fecha_div is not None:
                    fecha_div = fecha_div.text
                    fecha = fecha_div.split('-')[0].rstrip()
                    if 'Today' in fecha:
                        fecha = datetime.now().strftime("%d %b %Y")
                    elif 'Yesterday' in fecha:
                        fecha = (datetime.now() - timedelta(days=1)).strftime("%d %b %Y")
                divs2 = div.find_all('a', class_='next-m:flex next-m:!mt-0 ml-2 mt-2 min-h-[32px] w-full hover:cursor-pointer')

                for div2 in divs2:
                    partidos.append('https://www.oddsportal.com' + div2['href'])
                    equipos = div2.find_all('p', class_='participant-name truncate')
                    home_team.append(equipos[0].text)
                    away_team.append(equipos[1].text)
                    paginas2.append(i)
                    ligas.append(Nombres_ligas[i]) 
                    fechas.append(fecha)

                if next_button:
                    print(next_button)
                
                    next_button.click()
                next_button = None
                
    driver.quit()
    new_data_df = pd.DataFrame()        
    new_data_df['liga'] = ligas
    new_data_df['url'] = partidos
    new_data_df['Home_team'] = home_team
    new_data_df['Away_team'] = away_team
    new_data_df['pagina'] = paginas2
    new_data_df['fecha'] = fechas

    return new_data_df

In [28]:
dataset_urls = dataset_urls_ods2(['https://www.oddsportal.com/basketball/usa/nba/results/#/page/21/'], Nombres_ligas)

<selenium.webdriver.remote.webelement.WebElement (session="fdaec51230fcb9ab155860e6be5c17f7", element="f.E99BB15ED1322B9B3A3A5BF5F4C46692.d.079AA18811240ABF5032AB861EAD8E4D.e.207")>


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (1113, -513)
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00C7B5A3+24387]
	(No symbol) [0x00C05904]
	(No symbol) [0x00AE0753]
	(No symbol) [0x00B2EE50]
	(No symbol) [0x00B2D1F4]
	(No symbol) [0x00B2AD87]
	(No symbol) [0x00B2A044]
	(No symbol) [0x00B1E6D5]
	(No symbol) [0x00B4D84C]
	(No symbol) [0x00B1E164]
	(No symbol) [0x00B4DAC4]
	(No symbol) [0x00B6F138]
	(No symbol) [0x00B4D646]
	(No symbol) [0x00B1C59F]
	(No symbol) [0x00B1D8E4]
	GetHandleVerifier [0x00F7D883+3179043]
	GetHandleVerifier [0x00F96CF9+3282585]
	GetHandleVerifier [0x00F9167C+3260444]
	GetHandleVerifier [0x00D14330+650448]
	(No symbol) [0x00C0ED0D]
	(No symbol) [0x00C0BAF8]
	(No symbol) [0x00C0BC99]
	(No symbol) [0x00BFE530]
	BaseThreadInitThunk [0x767BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x776282AE+286]
	RtlGetAppContainerNamedObjectPath [0x7762827E+238]


In [35]:
url = 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/'
url_ods = [url + str(i) + '/' for i in range(1, 29)]
Nombres_ligas = ['NBA'] * 28
url_ods

['https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/1/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/2/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/3/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/4/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/5/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/6/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/7/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/8/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/9/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/10/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/11/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/results/#/page/12/',
 'https://www.oddsportal.com/basketball/usa/nba-2023-2024/res

In [22]:
dataset_urls

,liga,url,Home_team,Away_team,pagina,fecha
0,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Brooklyn Nets,Boston Celtics,0,15 Mar 2025
1,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Phoenix Suns,Sacramento Kings,0,15 Mar 2025
2,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Denver Nuggets,Los Angeles Lakers,0,15 Mar 2025
3,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Utah Jazz,Toronto Raptors,0,15 Mar 2025
4,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Houston Rockets,Dallas Mavericks,0,15 Mar 2025
5,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Memphis Grizzlies,Cleveland Cavaliers,0,15 Mar 2025
6,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Minnesota Timberwolves,Orlando Magic,0,15 Mar 2025
7,NBA,https://www.oddsportal.com/basketball/usa/nba/...,San Antonio Spurs,Charlotte Hornets,0,15 Mar 2025
8,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Atlanta Hawks,Los Angeles Clippers,0,15 Mar 2025
9,NBA,https://www.oddsportal.com/basketball/usa/nba/...,Miami Heat,Boston Celtics,0,15 Mar 2025


In [ ]:
def dataset_datos_ods(dataset_ods, dataset_urls):
    
    keys = ['liga', 'id', 'fecha','url','Home_team','Away_team','1_Average','X_Average','2_Average','1_Highest','X_Highest','2_Highest','Yes_Average','No_Average','Yes_Highest','No_Highest','1X_Average','12_Average','X2_Average',
        '1X_Highest','12_Highest','X2_Highest','over_0.5_Total_Goals','under_0.5_Total_Goals','over_.75_Total_Goals','under_.75_Total_Goals','over_ +1_Total_Goals','under_ +1_Total_Goals',
        'over_.25_Total_Goals','under_.25_Total_Goals','over_1.5_Total_Goals','under_1.5_Total_Goals','over_ +2_Total_Goals','under_ +2_Total_Goals','over_2.5_Total_Goals','under_2.5_Total_Goals',
        'over_ +3_Total_Goals','under_ +3_Total_Goals','under_.25_Total_Goals','over_3.5_Total_Goals','under_3.5_Total_Goals','over_ +4_Total_Goals','under_ +4_Total_Goals','over_4.5_Total_Goals',
        'under_4.5_Total_Goals','over_ +5_Total_Goals','under_ +5_Total_Goals','over_5.5_Total_Goals','under_5.5_Total_Goals','over_ +6_Total_Goals','under_ +6_Total_Goals','over_6.5_Total_Goals',
        'under_6.5_Total_Goals','over_7.5_Total_Goals','under_7.5_Total_Goals','over_8.5_Total_Goals','under_8.5_Total_Goals','over_3.5_Total_Goals','AH_-3.5_H','AH_-3.5_A','AH_3.25_H','AH_3.25_A',
        'AH_p -3_H','AH_p -3_A','AH_2.75_H','AH_2.75_A','AH_-2.5_H','AH_-2.5_A','AH_2.25_H','AH_2.25_A','AH_p -2_H','AH_p -2_A','AH_1.75_H','AH_1.75_A','AH_-1.5_H','AH_-1.5_A','AH_1.25_H',
        'AH_1.25_A','AH_p -1_H','AH_p -1_A','AH_0.75_H','AH_0.75_A','AH_-0.5_H','AH_-0.5_A','AH_0.25_H','AH_0.25_A','AH_ap 0_H','AH_ap 0_A','AH_+0.5_H','AH_+0.5_A','AH_p +1_H','AH_p +1_A',
        'AH_+1.5_H','AH_+1.5_A','AH_p +2_H','AH_p +2_A','AH_+2.5_H','AH_+2.5_A']
    extensiones = ['#1X2;2', '#bts;2', '#double;2', '#over-under;2', '#ah;2']
    columnas = [['1_Average','X_Average','2_Average','1_Highest','X_Highest','2_Highest'],['Yes_Average', 'No_Average', 'Yes_Highest', 'No_Highest'],
            ['1X_Average','12_Average','X2_Average','1X_Highest','12_Highest','X2_Highest']]
    
    dataset_urls['id'] = dataset_urls['fecha'].astype(str) + '_' + dataset_urls['liga'].astype(str) + '_' + dataset_urls['Home_team'].astype(str) + '_' + dataset_urls['Away_team'].astype(str)

    for index, row in dataset_urls.iterrows():

        if row['id'] not in dataset_ods['id'].values:

            print(index)
            diccionario_estadisticas = {key: None for key in keys}
            diccionario_estadisticas['liga'] = row['liga']
            diccionario_estadisticas['url'] = row['url']
            diccionario_estadisticas['Home_team'] = row['Home_team']
            diccionario_estadisticas['Away_team'] = row['Away_team']
            diccionario_estadisticas['id'] = row['id']
            diccionario_estadisticas['fecha'] = row['fecha']
            i = -1
            for extension in extensiones:
                
                i += 1
                url = row['url'] + extension

                options = Options()
                #options.add_experimental_option("detach", True)
                options.add_argument("--disable-search-engine-choice-screen")
                options.add_argument("--headless")

                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

                driver.get(url)
                driver.maximize_window()
                last_height = driver.execute_script("return document.body.scrollHeight")
                while True:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(3)  # Espera un poco para que se cargue el contenido dinámico
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    if new_height == last_height:
                        break
                    last_height = new_height
                    
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')

                if i == 0 or i == 1 or i == 2:
                    divs = soup.find_all('div', class_='border-black-borders flex min-w-[60px] flex-col items-center justify-center gap-1 border-l')
                    temp = []
                    for div in divs:
                        
                        if len(str(div)) == 240 or len(str(div)) == 241:
                            if div.find('p', class_='height-content').text[-1] != "%":
                                temp.append(div)
                    for j in range(len(temp)):
                        diccionario_estadisticas[columnas[i][j]] = temp[j].find('p', class_='height-content').text
                
                if i == 3 or i == 4:
                    divs = soup.find_all('div', class_='border-black-borders hover:bg-gray-light flex h-9 cursor-pointer border-b border-l border-r text-xs')
                    for j in range(len(divs)):
                        if i == 3:
                            temp1 = 'over_'+str(divs[j].find('p', class_ = 'max-sm:!hidden').text[-4:-1])+'_Total_Goals'
                            temp2 = 'under_'+str(divs[j].find('p', class_ = 'max-sm:!hidden').text[-4:-1])+'_Total_Goals'
                        elif i == 4:
                            temp1 = 'AH_'+str(divs[j].find('p', class_ = 'max-sm:!hidden').text[-5:-1])+'_H'
                            temp2 = 'AH_'+str(divs[j].find('p', class_ = 'max-sm:!hidden').text[-5:-1])+'_A'

                        if temp1 in diccionario_estadisticas.keys():
                            temp = divs[j].find_all('div', class_ = 'flex-center border-black-borders min-w-[60px] flex-col gap-1 pb-0.5 pt-0.5 relative')
                            diccionario_estadisticas[temp1] = temp[0].find('p', class_ = 'height-content !text-black-main next-m:min-w-[100%] flex-center min-h-full min-w-[50px] hover:!bg-gray-medium default-odds-bg-bgcolor border gradient-green-added-border').text
                            diccionario_estadisticas[temp2] = temp[1].find('p', class_ = 'height-content !text-black-main next-m:min-w-[100%] flex-center min-h-full min-w-[50px] hover:!bg-gray-medium default-odds-bg-bgcolor border gradient-green-added-border').text
            new_data_df = pd.DataFrame([diccionario_estadisticas])
            # Aqui debo incluir el formateo para que todo funcione correctamente
            dataset_ods = pd.concat([dataset_ods, new_data_df], ignore_index=True)
            dataset_ods.to_csv('Datos raw/Datos_cuotas.csv',index=False)

    return dataset_ods

In [8]:
dataset_ods = pd.read_csv('Datos raw/Datos_cuotas.csv')
dataset_ods = funciones.dataset_datos_ods(dataset_ods, dataset_urls)

FileNotFoundError: [Errno 2] No such file or directory: 'Datos raw/Datos_cuotas.csv'